# Projeto 2 - Ciência dos Dados

Nome: Nevton de Castro

Nome: Omar Dibo

Nome: Laura Perim 

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [80]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@Devsegundoseme1***

In [81]:
#Dados de autenticação do twitter: authpass

#Coloque aqui o identificador da conta no twitter: @Devsegundoseme1

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:
    data = json.load(fp)
    
#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [86]:
#Produto escolhido:
produto = 'civic'

#Quantidade mínima de mensagens capturadas:
n = 750
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't' and msg.full_text.lower()[2] != ' ':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
                break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs1[:n])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs1[n:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()



___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [42]:
civic = pd.read_excel('civic.xlsx')
spm1 = civic[civic.valor==0] #tweets de spam ou venda
irr1 = civic[civic.valor==1] #tweets irrelevantes
rel1 = civic[civic.valor==2] #tweets de elogios ou criticas
irr = irr1['Treinamento'].tolist()
rel = rel1['Treinamento'].tolist()
spm = spm1['Treinamento'].tolist()

In [43]:
spm1.valor.value_counts() #40
rel1.valor.value_counts() #172
irr1.valor.value_counts() #538

1    538
Name: valor, dtype: int64

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [44]:
#limpando os tweets relevantes

nop = [] 
#nao vamos remover "#" nem "|"
for i in rel:                    #elimando os pontos dos tweets
    punctuation = '[!\-.:?;,_+]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', i.lower())
    nop.append(text_subbed)
    
relevante_limpo = [] 

for i in nop:                    #eliminandno o produto dos tweets
    trash = 'civic'
    text_subbed = re.sub(trash, ' ', i)
    relevante_limpo.append(text_subbed)
    
relfinal = [] 
for i in relevante_limpo:         
    e = pd.Series(i.split())    #separa as palavras dos tweets
    for i in e:                 
        relfinal.append(i)      
df = pd.DataFrame(relfinal)     #cria um dataframe para analisar a frequencia de cada palavra
reldata = df[0].value_counts(True) 

#limpando os tweets irrelevantes

nop2 = []
for i in irr:                      #elimando os pontos dos tweets
    punctuation = '[!\-.:?;,_\+]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', i.lower())
    nop2.append(text_subbed)
    
irrelevante_limpo = []
for i in nop2:                     #eliminandno o produto dos tweets
    trash = 'civic'
    text_subbed = re.sub(trash, ' ', i)
    irrelevante_limpo.append(text_subbed)
relfinal2 = []
for i in irrelevante_limpo:        #separa as palavras dos tweets
    e = pd.Series(i.split()) 
    for i in e:
        relfinal2.append(i)
df2 = pd.DataFrame(relfinal2)  
irrdata = df2[0].value_counts(True)    

#limpando os tweets de spam

nop3 = []
#nao vamos remover "#" nem "|"
for i in spm:                        #elimando os pontos dos tweets
    punctuation = '[!\-.:?;,_\+]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', i.lower())
    nop3.append(text_subbed)
spam_limpo = []
for i in nop3:                       #eliminandno o produto dos tweets
    trash = 'civic'
    text_subbed = re.sub(trash, ' ', i)
    spam_limpo.append(text_subbed)
relfinal3 = []
for i in spam_limpo:                 #separa as palavras dos tweets
    e = pd.Series(i.split())
    for i in e:
        relfinal3.append(i)
df3 = pd.DataFrame(relfinal3)
spmdata = df3[0].value_counts(True)



In [45]:
k = []
oi = civic['Treinamento'].tolist()

for i in oi:
    
    
    s = i.split()
    
    for t in s:
        
        k.append(i)

a = list(set(k))
td = len(a)            #conta todas as palavras dos tweets

k=[]
for i in relevante_limpo:
    s = i.split()
    for t in s:
        k.append(i)
        
tdrel = len(k)        #conta todas as palavras dos tweets relevantes

k=[]
for i in irrelevante_limpo:
    s = i.split()
    for t in s:
        k.append(i)
        
tdirr = len(k)        #conta todas as palavras dos tweets irrelevantes

k=[]
for i in spam_limpo:
    s = i.split()
    for t in s:
        k.append(i)
        
tdspm = len(k)        #conta todas as palavras dos tweets de spam
print(tdrel)
print(tdirr)
print(tdspm)

2485
8382
1260


In [46]:
df3[0].value_counts(True)['pneus']

0.0007936507936507937

In [47]:
def Classificador_Naive_Bayes(tweets):  #classifica Naive-bayes
    
    lista =[]

    d=-1
    for i in tweets:  # i = cada tweet dentro de cada categoria (relevante,irrelevante,spam)
        t=1

        e = pd.Series(i.split())
        
        for w in e:                     # w = cada palvra dentro de cada tweet
            if w in df[0].tolist():
                r = df[0].value_counts(True)[w]     #basico, sem Laplace(ver proxima cell)
            else:
                r = 0                         
            t = r*t
        t*(172/750)        #numero de tweets relevantes/numero total de tweets
        o=[]
        o.append(t)         
        lista.append(o)


    d = -1
    for i in tweets:  # i = cada tweet dentro de cada categoria (relevante,irrelevante,spam)
        t =1
        d =d+1
        e = pd.Series(i.split())
        for w in e:                     # w = cada palvra dentro de cada tweet
            if w in df2[0].tolist():
                r = df2[0].value_counts(True)[w]    #basico, sem Laplace(ver proxima cell)
            else:
                r= 0
            t = r*t
        t*(538/750)                  #numero de tweets irrelevantes/numero total de tweets
        lista[d].append(t)

    d = -1
    for i in tweets:  
        t =1
        d =d+1
        e = pd.Series(i.split())
        for w in e:
            if w in df3[0].tolist():
                r = df3[0].value_counts(True)[w]    #basico, sem Laplace(ver proxima cell)
            else:
                r= 0
            t = r*t
        t*(40/750)                     #numero de tweets spam/numero total de tweets
        lista[d].append(t)
    qual=[]
    m=-1
    for i in lista:
        m=m+1
        if (lista[m][0] > lista[m][1]) and (lista[m][0] > lista[m][2]):
            qual.append('relevante')
        if (lista[m][1] > lista[m][0]) and (lista[m][1] > lista[m][2]):
            qual.append('irrelevante')
        if (lista[m][2] > lista[m][0]) and (lista[m][2] > lista[m][1]):
            qual.append('spam')
    return qual


In [48]:
def Classificador_Naive_Bayes_laplace(tweets):  #classifica Naive-bayes com Laplace smooting
    
    lista =[]

    d=-1
    for i in tweets:  # i = cada tweet dentro de cada categoria (relevante,irrelevante,spam)
        t=1

        e = pd.Series(i.split())
        
        for w in e:                     # w = cada palvra dentro de cada tweet
            if w in df[0].tolist():
                r = (df[0].value_counts()[w]+1)/(tdrel + td)   #Laplace smooting
            else:
                r = 1/(tdrel + td)                             #Laplace smooting
            t = r*t
        t*(172/750)        #numero de tweets relevantes/numero total de tweets
        o=[]
        o.append(t)         
        lista.append(o)


    d = -1
    for i in tweets:  # i = cada tweet dentro de cada categoria (relevante,irrelevante,spam)
        t =1
        d =d+1
        e = pd.Series(i.split())
        for w in e:                     # w = cada palvra dentro de cada tweet
            if w in df2[0].tolist():
                r = (df2[0].value_counts()[w]+1)/(tdirr + td)  #Laplace smooting
            else:
                r= 1/(tdirr + td)                              #Laplace smooting
            t = r*t
        t*(538/750)                  #numero de tweets irrelevantes/numero total de tweets
        lista[d].append(t)

    d = -1
    for i in tweets:  
        t =1
        d =d+1
        e = pd.Series(i.split())
        for w in e:
            if w in df3[0].tolist():
                r = (df3[0].value_counts()[w]+1)/(tdspm + td) #Laplace smooting
            else:
                r= 1/(tdspm+td)                               #Laplace smooting
            t = r*t
        t*(40/750)                     #numero de tweets spam/numero total de tweets
        lista[d].append(t)
    classificados=[]
    m=-1
    for i in lista:
        m=m+1
        if (lista[m][0] > lista[m][1]) and (lista[m][0] > lista[m][2]): #julga se e relevante
            classificados.append('relevante')
        if (lista[m][1] > lista[m][0]) and (lista[m][1] > lista[m][2]): #julga se e irrelevante
            classificados.append('irrelevante')
        if (lista[m][2] > lista[m][0]) and (lista[m][2] > lista[m][1]): #julga se e spam
            classificados.append('spam')
    return classificados


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [71]:

erros_rel = 0
for i in Classificador_Naive_Bayes_laplace(relevante_limpo):
    if i != 'relevante':
        erros_rel  += 1       #erros na classificao de relevantes

print('numero total de erros nos relevantes: {0}'.format(erros_rel))
r = int(100*erros_rel/172)
print('porcentagem de erros: {0}%'.format(r))

numero total de erros nos relevantes: 6
porcentagem de erros: 3%


In [72]:
erros_irr = 0
for i in Classificador_Naive_Bayes_laplace(irrelevante_limpo):
    if i != 'irrelevante':
        erros_irr += 1       #erros na classificao de irrelevantes

print('numero total de erros nos irrelevantes: {0}'.format(erros_irr))
i = int(100*erros_irr/538)
print('porcentagem de erros: {0}%'.format(i))

numero total de erros nos irrelevantes: 281
porcentagem de erros: 52%


In [73]:
erros_spm = 0
for i in Classificador_Naive_Bayes_laplace(spam_limpo):
    if i != 'spam':        
        erros_spm += 1        #erros na classificao de spam

print('numero total de erros nos spams: {0}'.format(erros_spm))
s = int(100*erros_spm/40)
print('porcentagem de erros: {0}%'.format(erros_spm))

numero total de erros nos spams: 0
porcentagem de erros: 0%


In [75]:
erros_total = erros_rel + erros_irr + erros_spm      #erros totais
print('todos os erros: {0}'.format(erros_total))
t = int(100*erros_total/750)
print('porcentagem de erros: {0}%'.format(t))

todos os erros: 287
porcentagem de erros: 38%


# Monitoramento periódico de análise de sentimento que fica rodando em loop e apresenta as informações  em um painel

In [79]:
teste = ['eu amo o civic ele e muito legal', 'vendo civic usado so chamar no zapzap', 'nossa eu amo o civic usado que minha mae roubou do meu pai depois do divorcio']
Classificador_Naive_Bayes_laplace(teste)

['relevante', 'spam', 'relevante']

In [111]:
#proof of concept

api = tweepy.API(auth)
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't' and msg.full_text.lower()[2] != ' ':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > 5:
            break


Classificador_Naive_Bayes_laplace(msgs)

['relevante', 'relevante', 'relevante', 'relevante', 'relevante']

In [ ]:
import time

while True:
    time.sleep(300)    #a cada 5 minutos 5 novos tweets sao avaliados
    api = tweepy.API(auth)
    i = 1
    msgs = []
    for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
        if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't' and msg.full_text.lower()[2] != ' ':
            msgs.append(msg.full_text.lower())
            i += 1
            if i > 5:
                print(msgs)
                print(Classificador_Naive_Bayes_laplace(msgs))
                break



___
### Concluindo

   Nesse trabalho analisamos 750 tweets, classificando entre 3 categorias (spam, irrelevantes e relevantes) a fim de criar um banco de dados que pode classificar futuros tweets utilizando o método de Naive Bayes. 
   Para refinar a classificação, removemos os pontos dos tweets, assim como o nome do produto (civic) analisado nesse projeto e também utilizamos o método de Laplace para aprimorar os resultados. 
   Para verificar a precisão das previsões, rodamos o programa nos itens já avaliados, com isso obtivemos os resultados de 3% de erro na classificação de tweets relevantes, 52% de erros na classificação de tweets irrelevantes e 0% de erro em tweets de spam, totalizando um total de 38% de erro.
   Por fim, Criamos um ‘while True’, para classificar novos tweets em tempo real (a cada 5 minutos).
   

#### O que poderia melhorar

   Para melhorar a precisão poderíamos ter mais tweets de base de dados e poderíamos adiciona-los para a base conforme o programa classifica esses novos tweets, assim entrando em machine learning. 

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**